In [ ]:
!pip install yfinance pandas numpy

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
cryptos = ["BTC-USD", "ETH-USD", "BNB-USD", "ADA-USD"]
all_data = []

for ticker in cryptos:
    df = yf.download(
        ticker,
        start="2020-01-01",
        auto_adjust=False,
        progress=False
    )

    # FIX MultiIndex columns
    df.columns = df.columns.get_level_values(0)

    df = df.reset_index()

    # Rename columns to MATCH SQL
    df = df.rename(columns={
        "Date": "trade_date",
        "Open": "open_price",
        "High": "high_price",
        "Low": "low_price",
        "Close": "close_price",
        "Adj Close": "adj_close_price",
        "Volume": "volume"
    })

    # Calculations
    df["daily_return"] = df["adj_close_price"].pct_change()
    df["sma_21"] = df["adj_close_price"].rolling(21).mean()
    df["sma_50"] = df["adj_close_price"].rolling(50).mean()
    df["volatility"] = df["daily_return"].rolling(21).std()

    df["trade_signal"] = np.where(df["sma_21"] > df["sma_50"], 1, 0)
    df["strategy_return"] = df["daily_return"] * df["trade_signal"]

    df["crypto"] = ticker.replace("-USD", "")

    # KEEP ONLY SQL COLUMNS (ORDER MATTERS)
    df = df[
        [
            "trade_date",
            "open_price",
            "high_price",
            "low_price",
            "close_price",
            "adj_close_price",
            "volume",
            "daily_return",
            "sma_21",
            "sma_50",
            "volatility",
            "trade_signal",
            "strategy_return",
            "crypto"
        ]
    ]

    all_data.append(df)

In [ ]:
final_df = pd.concat(all_data, ignore_index=True)

# Remove NaN rows
final_df = final_df.dropna()

# Ensure correct date format
final_df["trade_date"] = pd.to_datetime(final_df["trade_date"]).dt.date

final_df.head()

Price,trade_date,open_price,high_price,low_price,close_price,adj_close_price,volume,daily_return,sma_21,sma_50,volatility,trade_signal,strategy_return,crypto
49,2020-02-19,10143.798828,10191.675781,9611.223633,9633.386719,9633.386719,46992019710,-0.050149,9780.904343,8932.137070,0.025722,1,-0.050149,BTC
50,2020-02-20,9629.325195,9643.216797,9507.900391,9608.475586,9608.475586,44925260237,-0.002586,9785.641602,8980.303096,0.025373,1,-0.002586,BTC
51,2020-02-21,9611.782227,9723.014648,9589.743164,9686.441406,9686.441406,40930547513,0.008114,9801.637416,9034.322520,0.025095,1,0.008114,BTC
52,2020-02-22,9687.707031,9698.231445,9600.728516,9663.181641,9663.181641,35838025154,-0.002401,9814.509161,9080.688467,0.025105,1,-0.002401,BTC
53,2020-02-23,9663.318359,9937.404297,9657.791016,9924.515625,9924.515625,41185185761,0.027044,9842.135370,9130.965645,0.025646,1,0.027044,BTC


In [ ]:
final_df.to_csv(
    "crypto_data_sql_ready.csv",
    index=False
)

print("CSV CREATED SUCCESSFULLY")

CSV CREATED SUCCESSFULLY
